## Data Origin :: entsoe

The origin for this data is the transparency platform "entsoe". It's a central collection and publiaction of electricity generation, transportation and consumption data and information for the pan-European market. The url for the following data is: https://transparency.entsoe.eu/balancing/r2/imbalance/show


## Part 1: Data mining

## Imports

In [1]:
import datetime, time, os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pandas as pd
import glob
import json

from functools import reduce

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

from datetime import datetime, timezone, timedelta

resample_size = "15MIN"
resample_factor = 15

#windowfactor = 15

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# Function in this notebook

# Functions in this notebook

In [2]:
def to_datetime(df):
    '''
    We want the date and time to function as the index of the observations in a time-series.
    In this data frame the time column represent a period. We need to adapt this values to create a datetime format.  
    '''

    #Split the time period to a new temporary data frame.
    df[['dt_start_utc','Last']] = df.dt_start_utc.str.split("-", expand=True)

    #According to a correct datetime convertion, we need to consider the given format of the date in our data frame: DD-MM-YY.
    #In this situation the value of dayfirst must be set to "True".
    df["dt_start_utc"] = pd.to_datetime(df["dt_start_utc"], dayfirst=True)
    
    #The first date of each period will be assigned to an datetime format and set to index
    
    df.set_index('dt_start_utc', inplace=True)

    return(df)

In [3]:
def cleaningcolumns(df):
    '''This function simplify the column names of an given data frame.
        Special characters will replaced by an empty string and all 
        letters will be lowered. 
    '''  
    df.rename(str.lower, axis='columns', inplace=True)
    df.columns = df.columns.str.replace('(', '')
    df.columns = df.columns.str.replace(')', '')
    df.columns = df.columns.str.replace(' ','')
    df.columns = df.columns.str.replace('-','')
    df.columns = df.columns.str.replace('|','')
    df.columns = df.columns.str.replace(']','')
    df.columns = df.columns.str.replace('[','')
    df.columns = df.columns.str.replace('+','plus')
    df.columns = df.columns.str.replace('-','minus')
    df.columns = df.columns.str.replace('/','')
    
    return df

In [4]:
def mwh_sign(df):
    '''The columns representing the total imbalance in MWh are only given in absolute 
    values, which means that the mathematcial sign isn't represented in numbers, but 
    this information is given in the column "situation" through the values "Deficit",
    "Surplus" and "Balanced". We need to extract this string informations to numercial 
    informations. We can generate from this point the total imbalace price with the 
    correct mathematical sign.'''

    #Strings of the column "Situation" will be replaced by representive integer values.
    df.loc[df.situation == "Deficit", 'situation'] = int(-1)
    df.loc[df.situation == "Surplus", 'situation'] = int(1)
    df.loc[df.situation == "Balanced", 'situation'] = int(0)

    df["situation"] = df["situation"].astype('int64')

    #Checking, if all states are replaced
    print(df.situation.unique())

    return df

## Import German Imblance Data

We import the data, which is available in a single csv file, and assign it to a dataframe.

In [5]:
df_imbalance_de = pd.read_csv("../data/entsoe/df_imbalance_de.csv")

df_imbalance_de.shape

(17368, 8)

In [6]:
 #Find NaNs and duplicates in df_imbalance_de

print('There are {} missing values or NaNs in df_imbalance_de.'
      .format(df_imbalance_de.isnull().values.sum()))

temp_energy = df_imbalance_de.duplicated(keep='first').sum()

print('There are {} duplicate rows in df_imbalance_de based on all columns.'
      .format(temp_energy))

There are 17368 missing values or NaNs in df_imbalance_de.
There are 0 duplicate rows in df_imbalance_de based on all columns.


As we can see, the data frame has no duplicate values. Nevertheless, it has some NaNs and thus, we have to investigate further.

In [7]:
# Find the number of NaNs in each column
df_imbalance_de.isnull().sum(axis=0)

Imbalance settlement period (UTC)              0
+ Imbalance Price [EUR/MWh] - MBA|DE-LU        0
- Imbalance Price [EUR/MWh] - MBA|DE-LU        0
Status                                         0
Total Imbalance [MWh] - MBA|DE-LU              0
Difference                                 17368
Situation                                      0
Status.1                                       0
dtype: int64

We can drop the column "Difference", because it has NaNs in every row of this data frame. We will do this later.

In [8]:
df_imbalance_de.head()

,Imbalance settlement period (UTC),+ Imbalance Price [EUR/MWh] - MBA|DE-LU,- Imbalance Price [EUR/MWh] - MBA|DE-LU,Status,Total Imbalance [MWh] - MBA|DE-LU,Difference,Situation,Status.1
0,01.01.2021 00:00 - 01.01.2021 00:15,96.09,96.09,Final,52,NaN,Deficit,Intermediate
1,01.01.2021 00:15 - 01.01.2021 00:30,94.62,94.62,Final,7,NaN,Deficit,Intermediate
2,01.01.2021 00:30 - 01.01.2021 00:45,92.21,92.21,Final,13,NaN,Deficit,Intermediate
3,01.01.2021 00:45 - 01.01.2021 01:00,27.64,27.64,Final,38,NaN,Surplus,Intermediate
4,01.01.2021 01:00 - 01.01.2021 01:15,13.71,13.71,Final,17,NaN,Surplus,Intermediate


We want the date and time to function as the index of the observations in a time-series.

In this data frame the time column represent a period. We need to adapt this values to create a datetime format. 

In [9]:
df_imbalance_de.rename(columns={"Imbalance settlement period (UTC)": "dt_start_utc"}, inplace=True)
df_imbalance_de = to_datetime(df_imbalance_de).copy()


In [10]:
df_imbalance_de.head()

,+ Imbalance Price [EUR/MWh] - MBA|DE-LU,- Imbalance Price [EUR/MWh] - MBA|DE-LU,Status,Total Imbalance [MWh] - MBA|DE-LU,Difference,Situation,Status.1,Last
dt_start_utc,,,,,,,,
2021-01-01 00:00:00,96.09,96.09,Final,52,NaN,Deficit,Intermediate,01.01.2021 00:15
2021-01-01 00:15:00,94.62,94.62,Final,7,NaN,Deficit,Intermediate,01.01.2021 00:30
2021-01-01 00:30:00,92.21,92.21,Final,13,NaN,Deficit,Intermediate,01.01.2021 00:45
2021-01-01 00:45:00,27.64,27.64,Final,38,NaN,Surplus,Intermediate,01.01.2021 01:00
2021-01-01 01:00:00,13.71,13.71,Final,17,NaN,Surplus,Intermediate,01.01.2021 01:15


In [11]:
#Simplify the column names of the data frame.
df_imbalance_de = cleaningcolumns(df_imbalance_de)

In [12]:
df_imbalance_de.head()

,plusimbalancepriceeurmwhmbadelu,imbalancepriceeurmwhmbadelu,status,totalimbalancemwhmbadelu,difference,situation,status.1,last
dt_start_utc,,,,,,,,
2021-01-01 00:00:00,96.09,96.09,Final,52,NaN,Deficit,Intermediate,01.01.2021 00:15
2021-01-01 00:15:00,94.62,94.62,Final,7,NaN,Deficit,Intermediate,01.01.2021 00:30
2021-01-01 00:30:00,92.21,92.21,Final,13,NaN,Deficit,Intermediate,01.01.2021 00:45
2021-01-01 00:45:00,27.64,27.64,Final,38,NaN,Surplus,Intermediate,01.01.2021 01:00
2021-01-01 01:00:00,13.71,13.71,Final,17,NaN,Surplus,Intermediate,01.01.2021 01:15


In [13]:
df_imbalance_de.dtypes

plusimbalancepriceeurmwhmbadelu    float64
imbalancepriceeurmwhmbadelu        float64
status                              object
totalimbalancemwhmbadelu             int64
difference                         float64
situation                           object
status.1                            object
last                                object
dtype: object

In [14]:
#The mathematical sign is represented by a string value. The function convert the string to proper numerical format.
df_imbalance_de = mwh_sign(df_imbalance_de).copy()

#Adding the correct mathmatical sign to the total imbalance MWh value.
df_imbalance_de.eval("rz_saldo_mwh = situation * totalimbalancemwhmbadelu", inplace=True)

[-1  1  0]


The values of the columns "plusimbalancepriceeurmwhmbadelu" and "imbalancepriceeurmwhmbadelu" seems to be identical. If they're, we could drop one of them.

In [15]:
#If the difference of booths columns are always "0", booth columns are identical.
df_imbalance_de.eval("temp = plusimbalancepriceeurmwhmbadelu - imbalancepriceeurmwhmbadelu", inplace=True)
print(df_imbalance_de["temp"].unique())

[0.]


In [16]:
df_imbalance_de.columns

Index(['plusimbalancepriceeurmwhmbadelu', 'imbalancepriceeurmwhmbadelu',
       'status', 'totalimbalancemwhmbadelu', 'difference', 'situation',
       'status.1', 'last', 'rz_saldo_mwh', 'temp'],
      dtype='object')

We will remove the columns which will not be used at all in our analysis.

In [17]:
df_imbalance_de.drop(['imbalancepriceeurmwhmbadelu',
                      "status", "totalimbalancemwhmbadelu", "difference", 
                      "situation", "status.1", "last", "temp"], axis=1, inplace=True)



In [18]:
df_imbalance_de.head()

,plusimbalancepriceeurmwhmbadelu,rz_saldo_mwh
dt_start_utc,,
2021-01-01 00:00:00,96.09,-52
2021-01-01 00:15:00,94.62,-7
2021-01-01 00:30:00,92.21,-13
2021-01-01 00:45:00,27.64,38
2021-01-01 01:00:00,13.71,17


The column "plusimbalancepriceeurmwhmbadelu" represent the "regelzonenübergreifenden Bilanzausgleichsenergiepreise" short reBAP. To simplify the wording we like to rename the column.

In [19]:
df_imbalance_de.rename(columns={"plusimbalancepriceeurmwhmbadelu": "rebap_eur_mwh"},inplace=True)


In [20]:
df_imbalance_de.head()

,rebap_eur_mwh,rz_saldo_mwh
dt_start_utc,,
2021-01-01 00:00:00,96.09,-52
2021-01-01 00:15:00,94.62,-7
2021-01-01 00:30:00,92.21,-13
2021-01-01 00:45:00,27.64,38
2021-01-01 01:00:00,13.71,17


## Implement Real Life Time Delays to Data Frame

In real life, the balancing energie value is published with an delay of 30 mins. We remodeling this situation by shifting this column by two rows, which is equivalent to 30 mins.

In [21]:
df_imbalance_de["rz_saldo_mwh"] = df_imbalance_de["rz_saldo_mwh"].shift(periods=2)

In [22]:
df_imbalance_de.head()

,rebap_eur_mwh,rz_saldo_mwh
dt_start_utc,,
2021-01-01 00:00:00,96.09,NaN
2021-01-01 00:15:00,94.62,NaN
2021-01-01 00:30:00,92.21,-52.0
2021-01-01 00:45:00,27.64,-7.0
2021-01-01 01:00:00,13.71,-13.0


By shifting two periods of "rz_saldo_mwh", we created two NaNs. 

In [23]:
print("The Data Frame has",df_imbalance_de.isnull().sum().sum(),"missing values.")

The Data Frame has 2 missing values.


The data frame will be saved to a "pickle". This allows us to access the data frame with multiple notebooks.

In [24]:
df_imbalance_de.to_pickle('../data/pickle/df_imbalance_de.pickle')

## 1.1 Import Belgium

In [25]:
df = pd.read_csv("../data/entsoe/Imbalance_Belgien.csv")

In [26]:
 #Find NaNs and duplicates in data frame

print('There are {} missing values or NaNs in df_imbalance_de.'
      .format(df.isnull().values.sum()))

temp = df.duplicated(keep='first').sum()

print('There are {} duplicate rows in df_imbalance_de based on all columns.'
      .format(temp))

There are 0 missing values or NaNs in df_imbalance_de.
There are 0 duplicate rows in df_imbalance_de based on all columns.


In [27]:
df.head()

,dt_start_utc,+ Imbalance Price [EUR/MWh] - SCA|BE,- Imbalance Price [EUR/MWh] - SCA|BE,Status,Total Imbalance [MWh] - SCA|BE,Difference,Situation,Status.1
0,01.01.2021 00:00 - 01.01.2021 00:15,64.84,64.84,Final,16,-3,Deficit,Final
1,01.01.2021 00:15 - 01.01.2021 00:30,13.96,13.96,Final,38,2,Surplus,Final
2,01.01.2021 00:30 - 01.01.2021 00:45,12.40,12.40,Final,46,1,Surplus,Final
3,01.01.2021 00:45 - 01.01.2021 01:00,7.70,7.70,Final,65,14,Surplus,Final
4,01.01.2021 01:00 - 01.01.2021 01:15,64.25,64.25,Final,3,-9,Deficit,Final


In [28]:
df = to_datetime(df).copy()


In [29]:
cleaningcolumns(df)

,plusimbalancepriceeurmwhscabe,imbalancepriceeurmwhscabe,status,totalimbalancemwhscabe,difference,situation,status.1,last
dt_start_utc,,,,,,,,
2021-01-01 00:00:00,64.84,64.84,Final,16,-3,Deficit,Final,01.01.2021 00:15
2021-01-01 00:15:00,13.96,13.96,Final,38,2,Surplus,Final,01.01.2021 00:30
2021-01-01 00:30:00,12.40,12.40,Final,46,1,Surplus,Final,01.01.2021 00:45
2021-01-01 00:45:00,7.70,7.70,Final,65,14,Surplus,Final,01.01.2021 01:00
2021-01-01 01:00:00,64.25,64.25,Final,3,-9,Deficit,Final,01.01.2021 01:15
...,...,...,...,...,...,...,...,...
2021-07-29 11:15:00,-53.36,-53.36,Intermediate,42,18,Surplus,Intermediate,29.07.2021 11:30
2021-07-29 11:30:00,-106.41,-106.41,Intermediate,78,33,Surplus,Intermediate,29.07.2021 11:45
2021-07-29 11:45:00,-123.69,-123.69,Intermediate,74,37,Surplus,Intermediate,29.07.2021 12:00


In [30]:
#The mathematical sign is represented by a string value. The function convert the string to proper numerical format.
df = mwh_sign(df).copy()

#Adding the correct mathmatical sign to the total imbalance MWh value.
df.eval("rz_saldo_mwh_BE = situation * totalimbalancemwhscabe", inplace=True)

[-1  1  0]


In [31]:
df.head()

,plusimbalancepriceeurmwhscabe,imbalancepriceeurmwhscabe,status,totalimbalancemwhscabe,difference,situation,status.1,last,rz_saldo_mwh_BE
dt_start_utc,,,,,,,,,
2021-01-01 00:00:00,64.84,64.84,Final,16,-3,-1,Final,01.01.2021 00:15,-16
2021-01-01 00:15:00,13.96,13.96,Final,38,2,1,Final,01.01.2021 00:30,38
2021-01-01 00:30:00,12.40,12.40,Final,46,1,1,Final,01.01.2021 00:45,46
2021-01-01 00:45:00,7.70,7.70,Final,65,14,1,Final,01.01.2021 01:00,65
2021-01-01 01:00:00,64.25,64.25,Final,3,-9,-1,Final,01.01.2021 01:15,-3


We will remove the columns which will not be used at all in our analysis.

In [32]:
df.drop(["imbalancepriceeurmwhscabe", "status","totalimbalancemwhscabe", "difference", "status.1", "situation", "status.1", "last"], axis=1, inplace=True)
df.rename(columns={"plusimbalancepriceeurmwhscabe": "rebap_eur_mwh_BE"},inplace=True)

In [33]:
df.head()

,rebap_eur_mwh_BE,rz_saldo_mwh_BE
dt_start_utc,,
2021-01-01 00:00:00,64.84,-16
2021-01-01 00:15:00,13.96,38
2021-01-01 00:30:00,12.40,46
2021-01-01 00:45:00,7.70,65
2021-01-01 01:00:00,64.25,-3


According to a correct datetime convertion, we need to consider the given format of the date in our data frame: DD-MM-YY.
In this situation the value of dayfirst must be set to "True".

In [34]:
df.head()

,rebap_eur_mwh_BE,rz_saldo_mwh_BE
dt_start_utc,,
2021-01-01 00:00:00,64.84,-16
2021-01-01 00:15:00,13.96,38
2021-01-01 00:30:00,12.40,46
2021-01-01 00:45:00,7.70,65
2021-01-01 01:00:00,64.25,-3


In [35]:
count_inf = np.isinf(df_imbalance_de).values.sum()
count_nan = df_imbalance_de.isnull().sum().sum()
print("The data frame contains " + str(count_inf) + " infinite values and " + str(count_nan) + " missing values.")

The data frame contains 0 infinite values and 2 missing values.


### Implement Real Life Time Delays to Data Frame
In real life, the balancing energie value is published with an delay of 2 hours. We remodeling this situation by shifting this column by eight rows, which is equivalent to 2 hours.

In [36]:
df = df.shift(periods=8)

Copy data from a generic dataframe name (df) to more individual and specifc name.

In [37]:
df_be = df.copy()

## 1.2 Import Poland Data

We import the data, which is available in a single csv file, and assign it to a dataframe.

In [38]:
df = pd.read_csv("../data/entsoe/Imbalance_MB__Poland.csv")
df.shape

(5029, 8)

In [39]:
df.head()

,dt_start_utc,ImbalancePrice_PLN_MWh_MBA_PL,- Imbalance Price [PLN/MWh] - MBA|PL,Status,Total Imbalance [MWh] - MBA|PL,Difference,Situation,Status.1
0,01.01.2021 00:00 - 01.01.2021 01:00,191.33,191.33,Final,669,36,Surplus,Final
1,01.01.2021 01:00 - 01.01.2021 02:00,191.33,191.33,Final,521,15,Surplus,Final
2,01.01.2021 02:00 - 01.01.2021 03:00,191.33,191.33,Final,253,69,Surplus,Final
3,01.01.2021 03:00 - 01.01.2021 04:00,191.33,191.33,Final,130,47,Surplus,Final
4,01.01.2021 04:00 - 01.01.2021 05:00,191.33,191.33,Final,35,-85,Surplus,Final


In [40]:
 #Find NaNs and duplicates in data frame

print('There are {} missing values or NaNs in df_imbalance_de.'
      .format(df.isnull().values.sum()))

temp = df.duplicated(keep='first').sum()

print('There are {} duplicate rows in df_imbalance_de based on all columns.'
      .format(temp))

There are 0 missing values or NaNs in df_imbalance_de.
There are 0 duplicate rows in df_imbalance_de based on all columns.


In [41]:
df = to_datetime(df).copy()

In [42]:
#Simplify the column names of the data frame.
df = cleaningcolumns(df).copy()

In [43]:
df.head()

,imbalanceprice_pln_mwh_mba_pl,imbalancepriceplnmwhmbapl,status,totalimbalancemwhmbapl,difference,situation,status.1,last
dt_start_utc,,,,,,,,
2021-01-01 00:00:00,191.33,191.33,Final,669,36,Surplus,Final,01.01.2021 01:00
2021-01-01 01:00:00,191.33,191.33,Final,521,15,Surplus,Final,01.01.2021 02:00
2021-01-01 02:00:00,191.33,191.33,Final,253,69,Surplus,Final,01.01.2021 03:00
2021-01-01 03:00:00,191.33,191.33,Final,130,47,Surplus,Final,01.01.2021 04:00
2021-01-01 04:00:00,191.33,191.33,Final,35,-85,Surplus,Final,01.01.2021 05:00


In [44]:
df = mwh_sign(df).copy()
df.eval("rz_saldo_mwh_PL = situation * totalimbalancemwhmbapl", inplace=True)

[ 1 -1  0]


We will remove the columns which will not be used at all in our analysis.

In [45]:
df.drop(["imbalancepriceplnmwhmbapl", "status","totalimbalancemwhmbapl", "status.1", "situation", "difference", "last"], axis=1, inplace=True)
df.rename(columns={"imbalanceprice_pln_mwh_mba_pl": "rebap_eur_mwh_PL"},inplace=True)

In [46]:
df.head(10)

,rebap_eur_mwh_PL,rz_saldo_mwh_PL
dt_start_utc,,
2021-01-01 00:00:00,191.33,669
2021-01-01 01:00:00,191.33,521
2021-01-01 02:00:00,191.33,253
2021-01-01 03:00:00,191.33,130
2021-01-01 04:00:00,191.33,35
2021-01-01 05:00:00,191.33,-317
2021-01-01 06:00:00,191.33,-445
2021-01-01 07:00:00,191.33,-377
2021-01-01 08:00:00,191.33,141


The data are published with an one hour intervall. To unify the data with the other data frames it's necessary to resample from one hour to 15 min intervalls. The interpolated values are forward filled to prevent data leakage.

In [47]:
df = df.resample(resample_size).ffill()
df.head()

,rebap_eur_mwh_PL,rz_saldo_mwh_PL
dt_start_utc,,
2021-01-01 00:00:00,191.33,669
2021-01-01 00:15:00,191.33,669
2021-01-01 00:30:00,191.33,669
2021-01-01 00:45:00,191.33,669
2021-01-01 01:00:00,191.33,521


The data are published one hour delayed, so we shift "rz_saldo_mwh_PL four rows.

In [48]:
df = df.shift(periods=4)

In [49]:
count = np.isinf(df).values.sum()
print("The data frame contains " + str(count) + " infinite values")
print("The Data Frame has",df.isnull().sum().sum(),"missing values.")

The data frame contains 0 infinite values
The Data Frame has 8 missing values.


In [50]:
df.head()

,rebap_eur_mwh_PL,rz_saldo_mwh_PL
dt_start_utc,,
2021-01-01 00:00:00,NaN,NaN
2021-01-01 00:15:00,NaN,NaN
2021-01-01 00:30:00,NaN,NaN
2021-01-01 00:45:00,NaN,NaN
2021-01-01 01:00:00,191.33,669.0


In [51]:
df_pl = df.copy()

## 1.4 Import Austrian Data

In [52]:
df = pd.read_csv("../data/entsoe/AUS_Imba.csv")
df.shape

(20089, 8)

 First, we simplify all column names.

In [53]:
cleaningcolumns(df)

,dt_start_utc,plusimbalancepriceeurmwhscaat,imbalancepriceeurmwhscaat,status,totalimbalancemwhscaat,difference,situation,status.1
0,01.01.2021 00:00 - 01.01.2021 00:15,85.42,85.42,Estimated,34,NaN,NaN,Final
1,01.01.2021 00:15 - 01.01.2021 00:30,84.96,84.96,Estimated,40,NaN,NaN,Final
2,01.01.2021 00:30 - 01.01.2021 00:45,80.71,80.71,Estimated,24,NaN,NaN,Final
3,01.01.2021 00:45 - 01.01.2021 01:00,70.53,70.53,Estimated,14,NaN,NaN,Final
4,01.01.2021 01:00 - 01.01.2021 01:15,77.71,77.71,Estimated,32,NaN,NaN,Final
...,...,...,...,...,...,...,...,...
20084,29.07.2021 05:00 - 29.07.2021 05:15,13.48,13.48,Intermediate,8,5.0,Surplus,Intermediate
20085,29.07.2021 05:15 - 29.07.2021 05:30,86.98,86.98,Intermediate,28,2.0,Deficit,Intermediate
20086,29.07.2021 05:30 - 29.07.2021 05:45,100.00,100.00,Intermediate,45,2.0,Deficit,Intermediate
20087,29.07.2021 05:45 - 29.07.2021 06:00,86.98,86.98,Intermediate,30,3.0,Deficit,Intermediate


In [54]:
df = to_datetime(df).copy()

In [55]:
 #Find NaNs and duplicates in df_imbalance_de

print('There are {} missing values or NaNs in df_imbalance_de.'
      .format(df.isnull().values.sum()))

temp = df.duplicated(keep='first').sum()

print('There are {} duplicate rows in df_imbalance_de based on all columns.'
      .format(temp))

There are 34736 missing values or NaNs in df_imbalance_de.
There are 0 duplicate rows in df_imbalance_de based on all columns.


As we can see, the data frame has no duplicate values. Nevertheless, it has plenty NaNs we have to investigate further.

In [56]:
# Find the number of NaNs in each column
df.isnull().sum(axis=0)

plusimbalancepriceeurmwhscaat        0
imbalancepriceeurmwhscaat            0
status                               0
totalimbalancemwhscaat               0
difference                       17368
situation                        17368
status.1                             0
Last                                 0
dtype: int64

All NaNs are in the columns "difference" and "situation". We won't use "difference" in our further analysis, so we don't investigate in its NaNs. First, we take a look of all values of the column "situation".

In [57]:
df.drop("difference", axis=1, inplace=True)

print(df.situation.nunique())
print(df.situation.unique())

3
[nan 'Deficit' 'Surplus' 'Balanced']


In [58]:
df1 = df[df.isna().any(axis=1)]
print (df1)

                     plusimbalancepriceeurmwhscaat  imbalancepriceeurmwhscaat  \
dt_start_utc                                                                    
2021-01-01 00:00:00                          85.42                      85.42   
2021-01-01 00:15:00                          84.96                      84.96   
2021-01-01 00:30:00                          80.71                      80.71   
2021-01-01 00:45:00                          70.53                      70.53   
2021-01-01 01:00:00                          77.71                      77.71   
...                                            ...                        ...   
2021-06-30 20:45:00                         123.85                     123.85   
2021-06-30 21:00:00                          26.43                      26.43   
2021-06-30 21:15:00                          20.93                      20.93   
2021-06-30 21:30:00                          11.40                      11.40   
2021-06-30 21:45:00         

When "situation" is NaN, "totalimbalancemwhscaat" has booth, positive and negative values. The orgin use of "situtation" is obsolete. We set this NaNs to 1 and apply the data frame to the function "mwh_sign".

In [59]:
df.columns

Index(['plusimbalancepriceeurmwhscaat', 'imbalancepriceeurmwhscaat', 'status',
       'totalimbalancemwhscaat', 'situation', 'status.1', 'Last'],
      dtype='object')

In [60]:
df.situation = df.situation.fillna(value=1)
df = mwh_sign(df).copy()

df.eval("rz_saldo_mwh_AUS = situation * totalimbalancemwhscaat", inplace=True)


df.drop(["plusimbalancepriceeurmwhscaat", "status","totalimbalancemwhscaat", "status.1", "situation", "Last"], axis=1, inplace=True)
df.rename(columns={"imbalancepriceeurmwhscaat": "rebap_eur_mwh_AUS"},inplace=True)

[ 1 -1  0]


In [61]:
df.head()

,rebap_eur_mwh_AUS,rz_saldo_mwh_AUS
dt_start_utc,,
2021-01-01 00:00:00,85.42,34
2021-01-01 00:15:00,84.96,40
2021-01-01 00:30:00,80.71,24
2021-01-01 00:45:00,70.53,14
2021-01-01 01:00:00,77.71,32


We need to shift the data 10 rows, because the data are usually 2,5 hours delayed  

In [62]:
df = df.shift(periods=10)

In [63]:
df_aus = df.copy()

## 1.5 Import Danemark

We import the data, which is available in a single csv file, and assign it to a dataframe.

In [64]:
df = pd.read_csv("../data/entsoe/Imbalance_MB__DK.csv")
df.shape

(5028, 9)

In [65]:
df = to_datetime(df).copy()

In [66]:
df = cleaningcolumns(df).copy()

In [67]:
 #Find NaNs and duplicates in df

print('There are {} missing values or NaNs in data frame.'
      .format(df.isnull().values.sum()))

temp = df.duplicated(keep='first').sum()

print('There are {} duplicate rows in df based on all columns.'
      .format(temp))

There are 10088 missing values or NaNs in data frame.
There are 0 duplicate rows in df based on all columns.


In [68]:
# Find the number of NaNs in each column
df.isnull().sum(axis=0)

generationplusimbalancepriceeurmwhmbadk2       8
generationimbalancepriceeurmwhmbadk2           8
priceforconsumptioneurmwhmbadk2                8
status                                         8
totalimbalancemwhmbadk2                        0
difference                                  5028
situation                                   5028
status.1                                       0
last                                           0
dtype: int64

When "situation" is NaN, "totalimbalancemwhscaat" has booth, positive and negative values. The orgin use of "situtation" is obsolete. We set this NaNs to 1 and apply the data frame to the function "mwh_sign".

In [69]:
df.situation = df.situation.fillna(value=1)
df = mwh_sign(df).copy()

df.eval("rz_saldo_mwh_dk2 = situation * totalimbalancemwhmbadk2", inplace=True)


df.drop(["last", "status", "status.1", "situation", "difference"], axis=1, inplace=True)
df.rename(columns={"generationimbalancepriceeurmwhmbadk2": "rebap_eur_mwh_dk2"},inplace=True)

[1]


In [70]:
df.head()

,generationplusimbalancepriceeurmwhmbadk2,rebap_eur_mwh_dk2,priceforconsumptioneurmwhmbadk2,totalimbalancemwhmbadk2,rz_saldo_mwh_dk2
dt_start_utc,,,,,
2021-01-01 00:00:00,48.19,48.19,48.19,141,141
2021-01-01 01:00:00,44.68,44.68,44.68,148,148
2021-01-01 02:00:00,42.92,42.92,42.92,156,156
2021-01-01 03:00:00,40.39,40.39,40.39,196,196
2021-01-01 04:00:00,40.20,40.20,40.20,216,216


The data are published with an one hour intervall. To unify the data with the other data frames it's necessary to resample from one hour to 15 min intervalls. The interpolated values are forward filled to prevent data leakage.

In [71]:
df = df.resample(resample_size).ffill();

We need to forward rows for 2.5 hours, because it's released with a delay.

In [72]:
df = df.shift(periods=10)

In [73]:
df.head()

,generationplusimbalancepriceeurmwhmbadk2,rebap_eur_mwh_dk2,priceforconsumptioneurmwhmbadk2,totalimbalancemwhmbadk2,rz_saldo_mwh_dk2
dt_start_utc,,,,,
2021-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN
2021-01-01 00:15:00,NaN,NaN,NaN,NaN,NaN
2021-01-01 00:30:00,NaN,NaN,NaN,NaN,NaN
2021-01-01 00:45:00,NaN,NaN,NaN,NaN,NaN
2021-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN


In [74]:
df_dk = df.copy()

## 1.6 Import Danemark DK1

In [75]:
df = pd.read_csv("../data/entsoe/Imbalance_MB__DK_DK1.csv")
df.shape

(5028, 9)

In [76]:
df = to_datetime(df).copy()

In [77]:
df = cleaningcolumns(df).copy()

In [78]:
df.drop(["last", "status", "status.1", "situation", "difference"], axis=1, inplace=True)

In [79]:
df.head()

,generationplusimbalancepriceeurmwhmbadk1,generationimbalancepriceeurmwhmbadk1,priceforconsumptioneurmwhmbadk1,totalimbalancemwhmbadk1
dt_start_utc,,,,
2021-01-01 00:00:00,48.19,48.19,48.19,75
2021-01-01 01:00:00,44.68,44.68,44.68,86
2021-01-01 02:00:00,42.92,42.92,42.92,30
2021-01-01 03:00:00,40.39,40.39,40.39,84
2021-01-01 04:00:00,40.20,40.20,40.20,30


The data are published with an one hour intervall. To unify the data with the other data frames it's necessary to resample from one hour to 15 min intervalls. The interpolated values are forward filled to prevent data leakage.

In [80]:
df = df.resample(resample_size).ffill();

We need to forward rows equivalent 2.5 hours, because it's released with a delay.

In [81]:
df = df.shift(periods=10)

In [82]:
df.head()

,generationplusimbalancepriceeurmwhmbadk1,generationimbalancepriceeurmwhmbadk1,priceforconsumptioneurmwhmbadk1,totalimbalancemwhmbadk1
dt_start_utc,,,,
2021-01-01 00:00:00,NaN,NaN,NaN,NaN
2021-01-01 00:15:00,NaN,NaN,NaN,NaN
2021-01-01 00:30:00,NaN,NaN,NaN,NaN
2021-01-01 00:45:00,NaN,NaN,NaN,NaN
2021-01-01 01:00:00,NaN,NaN,NaN,NaN


Copy the data frame to an unique and reasonable variable name

In [83]:
df_dk_1 = df.copy()

## Data Merge 

All data frames will be merged to a single data frame

In [84]:
dfs = [df_imbalance_de, df_be, df_pl, df_aus, df_dk, df_dk_1]

df_merged_2 = reduce(lambda  left,right: pd.merge(left, right, on=['dt_start_utc'],
                                            how='inner'), dfs)

In [85]:
df_merged_2.head()

,rebap_eur_mwh,rz_saldo_mwh,rebap_eur_mwh_BE,rz_saldo_mwh_BE,rebap_eur_mwh_PL,rz_saldo_mwh_PL,rebap_eur_mwh_AUS,rz_saldo_mwh_AUS,generationplusimbalancepriceeurmwhmbadk2,rebap_eur_mwh_dk2,priceforconsumptioneurmwhmbadk2,totalimbalancemwhmbadk2,rz_saldo_mwh_dk2,generationplusimbalancepriceeurmwhmbadk1,generationimbalancepriceeurmwhmbadk1,priceforconsumptioneurmwhmbadk1,totalimbalancemwhmbadk1
dt_start_utc,,,,,,,,,,,,,,,,,
2021-01-01 00:00:00,96.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 00:15:00,94.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 00:30:00,92.21,-52.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 00:45:00,27.64,-7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 01:00:00,13.71,-13.0,NaN,NaN,191.33,669.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
df_merged_2.shape

(17368, 17)

In [87]:
df_merged_2.tail()

,rebap_eur_mwh,rz_saldo_mwh,rebap_eur_mwh_BE,rz_saldo_mwh_BE,rebap_eur_mwh_PL,rz_saldo_mwh_PL,rebap_eur_mwh_AUS,rz_saldo_mwh_AUS,generationplusimbalancepriceeurmwhmbadk2,rebap_eur_mwh_dk2,priceforconsumptioneurmwhmbadk2,totalimbalancemwhmbadk2,rz_saldo_mwh_dk2,generationplusimbalancepriceeurmwhmbadk1,generationimbalancepriceeurmwhmbadk1,priceforconsumptioneurmwhmbadk1,totalimbalancemwhmbadk1
dt_start_utc,,,,,,,,,,,,,,,,,
2021-06-30 20:45:00,119.06,-79.0,104.51,-14.0,351.15,712.0,-47.04,-102.0,98.97,268.93,268.93,220.0,220.0,98.97,268.93,268.93,100.0
2021-06-30 21:00:00,3.10,-91.0,116.67,-50.0,358.31,343.0,-97.86,-87.0,98.97,268.93,268.93,220.0,220.0,98.97,268.93,268.93,100.0
2021-06-30 21:15:00,125.68,-57.0,119.98,-49.0,358.31,343.0,-199.94,-71.0,98.97,268.93,268.93,220.0,220.0,98.97,268.93,268.93,100.0
2021-06-30 21:30:00,100.92,4.0,104.51,-9.0,358.31,343.0,171.06,56.0,46.00,97.02,46.00,97.0,97.0,46.00,97.02,46.00,39.0
2021-06-30 21:45:00,-19.88,-62.0,104.51,-15.0,358.31,343.0,175.22,86.0,46.00,97.02,46.00,97.0,97.0,46.00,97.02,46.00,39.0


In [88]:
count_inf = np.isinf(df_merged_2).values.sum()
count_nan = df_merged_2.isnull().sum().sum()
print("The data frame contains " + str(count_inf) + " infinite values and " + str(count_nan) + " missing values.")

The data frame contains 0 infinite values and 136 missing values.


## Save Data Set

For further work we save the merged data frame as a "pickle".

In [89]:
df_merged_2.to_pickle('../data/pickle/df_merged_2.pickle')